### Process Embeddings and Convert to DataFrame

Also combine any additional embeddings generated from custom pdb requests

<div>UPDATE (as of 5 AM Friday -> Saturday): </div> This notebook is now obsolete on account of an inability to process all the sliced datums. The full CHROMA dataset is too large to be properly handled right now.

In [47]:
# Run once cell (because of location of notebook, i.e. not in base directory)

%load_ext autoreload
%autoreload 2

import os
os.chdir('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys

# Add the parent directory to the path so we can import the modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import pickle
import json

import time


from helpers.database import populate_representations, get_column, get_scalars, whatis
from helpers.edges import connect_edges, CascadingEdges
from helpers.cascades import Cascade, MakeCascade, Metrics, MetricsPair, MakeMetricsPair
from helpers.load_data import LoadData, save_df, save_edges


from functools import partial
from collections import defaultdict
from tqdm import tqdm
import jax.numpy as jnp
from jax import vmap, lax

from moleculib.assembly.dataset import ChromaDataset



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# FOLDER_PREAMBLE = "../scripts/"
FOLDER_PREAMBLE = "../data"


REDUCED_FOLDER = FOLDER_PREAMBLE + "/" + "denim-energy-1008-embeddings"
FULL_FOLDER = FOLDER_PREAMBLE + "/" + "denim-energy-full-CHROMA"

# General file path names
embeddings_file = "encoded_dataset.pkl"
sliced_proteins_file = "sliced_dataset.pkl"
tsne_file = "encoded_dataset_tsne.json"


chroma_reduced = LoadData(REDUCED_FOLDER, embeddings_file, sliced_proteins_file, tsne_file)
chroma_full = LoadData(FULL_FOLDER, embeddings_file, sliced_proteins_file)

start_time = time.time()
print("Loading reduced data")
chroma_reduced.load_all()
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print("Loading full data")
chroma_full.load_all()
print("--- %s seconds ---" % (time.time() - start_time))

Loading reduced data
--- 8.693283796310425 seconds ---
Loading full data


EOFError: Ran out of input

In [1]:
import pickle

# Function to load the sliced dataset
def load_sliced_dataset(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)
        
test_sliced = load_sliced_dataset("../../../ophiuchus/scripts/saved_embeddings/denim-energy-full-CHROMA/sliced_dataset.pkl")

print("Sliced datasets loaded successfully.")


EOFError: Ran out of input

In [2]:
test_sliced

NameError: name 'test_sliced' is not defined

### Chroma Reduced

In [20]:
# Make objects
reps_reduced, _ = populate_representations(chroma_reduced.encoded_dataset, chroma_reduced.sliced_dataset, chroma_reduced.tsne_data)
df_reduced = reps_reduced.to_dataframe()
print(f"Loaded full dataset: {df_reduced.shape}")

# Calculate the number of nodes per level
nodes_per_level_reduced = df_reduced.groupby('level').size()
print("Number of nodes per level:")
display(nodes_per_level_reduced)

# Count the "None" datums
n_none_datums_reduced = df_reduced[df_reduced['datum'].isnull()].shape[0]
print(f"Number of None datums: {n_none_datums_reduced}")



Loaded full dataset: (431465, 7)
Number of nodes per level:


level
0    221142
1    111222
2     56215
3     28602
4     14284
dtype: int64

Number of None datums: 5375


### Chroma Full

In [ ]:
%%time

# Make objects
reps_full, _ = populate_representations(chroma_full.encoded_dataset, chroma_full.sliced_dataset)
df_full = reps_full.to_dataframe()
print(f"Loaded full dataset: {df_full.shape}")

# Calculate the number of nodes per level
nodes_per_level_full = df_full.groupby('level').size()
print("Number of nodes per level:")
display(nodes_per_level_full)

# Count the "None" datums
n_none_datums_full = df_full[df_full['datum'].isnull()].shape[0]
print(f"Number of None datums: {n_none_datums_full}")


Loaded full dataset: (431465, 7)
Number of nodes per level:


level
0    221142
1    111222
2     56215
3     28602
4     14284
dtype: int64

Number of None datums: 5375
CPU times: user 1.94 s, sys: 61.5 ms, total: 2 s
Wall time: 2.03 s
